## DATA 311 - Fall 2020
### Assignment 5 - Due Weds, Nov 11 by midnight

For this assignment, 

- Get all of our sales rep rules/data loaded into the database
- Also load the October 2020 sales data
- Produce a few summaries!

In [11]:
import pandas as pd
import sqlite3

In [12]:
conn = sqlite3.connect('./data/RepStore.db')
curs = conn.cursor()

In [13]:
pd.read_sql("SELECT * FROM sqlite_master;", conn)

,type,name,tbl_name,rootpage,sql
0,table,tProd,tProd,2,CREATE TABLE tProd (\n prod_id INTE...
1,table,tRep,tRep,3,CREATE TABLE tRep(\n rep_id INTEGER...
2,table,tRepByReg,tRepByReg,4,CREATE TABLE tRepByReg(\n reg TEXT ...
3,index,sqlite_autoindex_tRepByReg_1,tRepByReg,5,None
4,table,tDivToReg,tDivToReg,6,CREATE TABLE tDivToReg(\n div TEXT ...
5,index,sqlite_autoindex_tDivToReg_1,tDivToReg,7,None
6,table,tRepByDiv,tRepByDiv,8,CREATE TABLE tRepByDiv(\n div TEXT ...
7,index,sqlite_autoindex_tRepByDiv_1,tRepByDiv,9,None
8,table,tState,tState,10,CREATE TABLE tState (\n st TEXT PRI...
9,index,sqlite_autoindex_tState_1,tState,11,None


---
1) What are our total sales for all data in the database, grouped by sales rep?

Return two columns:
- Sales rep name
- Total sales

In [14]:
pd.read_sql("""SELECT order_id, sum(qty*unit_price) AS total_sales
            FROM tOrderDetail
            JOIN tProd USING(prod_id)
            GROUP BY order_id;""", conn)

,order_id,total_sales
0,1,29.94
1,2,502.11
2,3,479.43
3,4,4171.87
4,5,346.93
...,...,...
4438,4439,296.45
4439,4440,121.56
4440,4441,154.89
4441,4442,711.43


In [15]:
pd.read_sql("""SELECT cust_id, sum(total_sales) AS total_sales
            FROM (SELECT order_id, sum(qty*unit_price) AS total_sales
                FROM tOrderDetail
                JOIN tProd USING(prod_id)
                GROUP BY order_id)
            JOIN tOrder USING(order_id)
            GROUP BY cust_id;""", conn)

,cust_id,total_sales
0,1,19205.84
1,2,40714.42
2,3,42760.22
3,4,73818.36
4,5,58460.04
...,...,...
305,306,49138.96
306,307,47967.05
307,308,37116.90
308,309,15223.48


In [16]:
pd.read_sql("""SELECT rep_name, sum(total_sales)
            FROM (SELECT cust_id, sum(total_sales) AS total_sales
                FROM (SELECT order_id, sum(qty*unit_price) AS total_sales
                    FROM tOrderDetail
                    JOIN tProd USING(prod_id)
                    GROUP BY order_id)
                JOIN tOrder USING(order_id)
                GROUP BY cust_id)
            JOIN tRepByCust USING(cust_id)
            JOIN tRep USING(rep_id)
            GROUP BY rep_id;""", conn)

,rep_name,sum(total_sales)
0,Frank,14932.25
1,Edgar,210914.70
2,Bob,1897559.54
3,Cathy,5045078.42
4,Alice,1483225.43
5,Diane,2825697.35


---
2) How many customers are assigned to each sales rep?

Return two columns:

- Sales rep name
- Number of customers assigned

In [17]:
pd.read_sql("""SELECT rep_id, count(cust_id) AS number_of_cust
            FROM tRepByCust
            GROUP BY rep_id;""", conn)

,rep_id,number_of_cust
0,1,1
1,2,6
2,3,55
3,4,135
4,5,41
5,6,72


In [18]:
pd.read_sql("""SELECT rep_name,number_of_cust
            FROM (SELECT rep_id, count(cust_id) AS number_of_cust
                FROM tRepByCust
                GROUP BY rep_id)
            JOIN tRep USING(rep_id);""", conn)

,rep_name,number_of_cust
0,Frank,1
1,Edgar,6
2,Bob,55
3,Cathy,135
4,Alice,41
5,Diane,72


---
3) What are our total sales for all data in the database, grouped by region?

Return two columns:

- Region
- Total sales

In [19]:
pd.read_sql("""SELECT reg, sum(total_sales)
            FROM (SELECT cust_id, sum(total_sales) AS total_sales
                FROM (SELECT order_id, sum(qty*unit_price) AS total_sales
                    FROM tOrderDetail
                    JOIN tProd USING(prod_id)
                    GROUP BY order_id)
                JOIN tOrder USING(order_id)
                GROUP BY cust_id)
            JOIN tCust USING(cust_id)
            JOIN tZip USING(zip)
            JOIN tState USING(st)
            JOIN tDivToReg USING(div)
            GROUP BY reg;""", conn)

,reg,sum(total_sales)
0,Midwest,2510094.60
1,Northeast,1920797.59
2,PR,189445.46
3,South,4016440.44
4,West,2840629.60
